In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
from mne.filter import next_fast_len
import os 
import joblib
import emd
import re
import scipy
import h5io

import warnings
import numpy as np
import re
import difflib 
from mne.preprocessing.bads import _find_outliers
from scipy.stats import kurtosis, zscore
import neurodsp
import mne
from glob import glob
import pandas as pd
from mne.filter import next_fast_len
from scipy.signal import hilbert, find_peaks, peak_widths, convolve
import Levenshtein as lev
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append('/Users/christinamaher/Documents/GitHub/LFPAnalysis')

In [3]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

In [ ]:
subject = 'MS011'

In [ ]:
base_dir = '/Users/christinamaher/Desktop/fellowship_data' # this is the root directory for most un-archived data and results 

save_dir = f'{base_dir}/{subject}/Ephys'  # save intermediate results in the 'work' directory
    
# I have saved most of my raw data in the 'projects directory'
behav_dir = f'{base_dir}/{subject}/Behavior'
neural_dir = f'{base_dir}/{subject}/Ephys'
anat_dir = f'{base_dir}/{subject}/Anat'
edf_files = glob(f'{neural_dir}/*.edf')
tfr_dir = save_dir

In [ ]:
mne_data_reref = mne.io.read_raw_fif(f'{base_dir}/{subject}/Ephys/wm_ref_ieeg.fif', preload=True)
photodiode = mne.io.read_raw_fif(f'{base_dir}/{subject}/Ephys/photodiode.fif', preload=True)


#behavior data 
behav_df = pd.read_csv(f'{base_dir}/{subject}/Behavior/{subject}_clean.csv')
beh_ts = behav_df['choice_ts'] #timestamp that should be aligned to photodiode 

# add column of ITI ts
behav_df['iti_ts'] = behav_df['choice_ts'] + 1500

#anat recon file
anat_file = glob(f'{anat_dir}/{subject}_labels.csv')[0]
elec_locs = pd.read_csv(anat_file)

In [ ]:
# load the photodiode and resample to match the neural data
photodiode = mne.io.read_raw_fif(f'{base_dir}/{subject}/Ephys/photodiode.fif', preload=True)
resample_sr = 500
photodiode.resample(sfreq=resample_sr, npad='auto', n_jobs=-1)

In [ ]:
# load behavior data and save timestamp(s) of interest as variable 
behavior = pd.read_csv(f'{base_dir}/{subject}/Behavior/{subject}_clean.csv')
choice_ts = np.array(behavior['choice_ts'])

In [ ]:
# plot photodiode and choice timestamps before alignment
%matplotlib qt
plt.plot(photodiode._data[0])
plt.xlabel("Time")
plt.ylabel("V")
plt.title("Photodiode")

zeros = np.array([0.05] * len(choice_ts))
x_ts = choice_ts
y_ts = zeros.T
plt.scatter(x_ts,y_ts,color='red')

plt.show()

In [ ]:
# Sample time-frequency data as an example
time_frequency_data = photodiode._data[0]

# Define the threshold values
threshold_min = -0.15

photodiode_deflected = []

# Detect sequential peaks
for value in time_frequency_data:
    if value > threshold_min:
        photodiode_deflected.append(1)
    elif value < threshold_min:
        photodiode_deflected.append(-1)

In [ ]:
def sum_sequential_values(arr):
    if not arr:
        return []

    result = []
    current_sum = arr[0]

    for i in range(1, len(arr)):
        if arr[i] == arr[i - 1]:
            current_sum += arr[i]
        else:
            result.append(current_sum)
            current_sum = arr[i]

    # Append the last calculated sum
    result.append(current_sum)

    return result

In [ ]:
lengths = sum_sequential_values(photodiode_deflected)

In [ ]:
def assign_peak_indices(lengths):
    photodiode_indices = []
    
    for l in lengths:
        if l < 0:
            l_temp = l * -1 # flip sign
            drop_numbers = np.ones(l_temp, dtype=int)
            photodiode_indices.append(drop_numbers)
        elif (l > 800) | (l < 700):
            drop_numbers = np.ones(l, dtype=int)
            photodiode_indices.append(drop_numbers)
        elif (1 < 800) & (l > 700): 
            keep_numbers = np.zeros(l, dtype=int)
            photodiode_indices.append(keep_numbers)
        else:
            print("error")
    
    return np.concatenate(photodiode_indices)

In [ ]:
photodiode_indices = assign_peak_indices(lengths)

In [ ]:
new_diode = photodiode._data[0].copy()
peaks_to_exclude = [index for index, value in enumerate(photodiode_indices) if value == 1]
new_diode[peaks_to_exclude] = np.min(photodiode._data[0]) # make these all super small

photodiode_final = photodiode.copy()
photodiode_final._data[0] = new_diode

In [ ]:
# plot photodiode
plt.plot(photodiode_final._data[0])
plt.xlabel("Time")
plt.ylabel("V")
plt.title("Photodiode")
plt.show()

In [ ]:
slope, offset = sync_utils.synchronize_data(choice_ts, 
                                            photodiode_final, 
                                            smoothSize=11, windSize=10, height=0.7)
print(slope,offset) # should be close to 1.0

In [ ]:
choice_ts = choice_ts * slope + offset

%matplotlib qt
plt.plot(photodiode_final._data[0])
plt.xlabel("Time")
plt.ylabel("V")
plt.title("Photodiode")

zeros = np.array([-0.15] * len(choice_ts))
x_ts = choice_ts * 500
y_ts = zeros.T
plt.scatter(x_ts,y_ts,color='red')

plt.show()

In [ ]:
# Create a dictionary with your event name (matching your dataframe), and the time-window for the event
evs = {'choice_ts': [-1.5, 1.5], # 1.5 seconds before choice, to 1.5 seconds after (full reward presentation)
       'iti_ts': [0.0, 0.5]} # first half a second of ITI (due to the jitter, the ITI ranges from 0.5 to 1.5s)

epochs_all_evs = {f'{x}': np.nan for x in evs}

In [ ]:
IED_args = {'peak_thresh':4,
           'closeness_thresh':0.25, 
           'width_thresh':0.2}


for event in evs.keys():
    # Make the epochs. 
    ev_epochs = lfp_preprocess_utils.make_epochs(load_path=f'{neural_dir}/wm_ref_ieeg.fif', 
                 slope=slope, offset=offset,
                 behav_name=event, behav_times=behav_df[event].values, 
                 ev_start_s=evs[event][0], ev_end_s=evs[event][1], buf_s = 1.0, IED_args=IED_args) #1.0 buf unsaved

    epochs_all_evs[event] = ev_epochs

In [ ]:
behav_params = ['r', 'acquired', 'condition','rd','ev','rpe']

In [ ]:
for event in evs.keys():

    event_metadata = epochs_all_evs[event].metadata.copy()
    
    #independent vars
    for param in behav_params: 
        event_metadata[param] = behav_df[param].tolist()

    epochs_all_evs[event].metadata = event_metadata 

In [ ]:
epochs_all_evs[event].metadata

Save raw epoched data

In [ ]:
for event in evs.keys():
    epochs_all_evs[event].save(f'{neural_dir}/epoch_{event}.fif', overwrite=True)

Baselining 

In [ ]:
# Set some spectrogram parameters 
freqs = np.logspace(*np.log10([2, 200]), num=30)
n_cycles = 3
sr = 500.0 
buf = 1.0
buf_ix = int(buf*sr)

In [ ]:
# Explicitly define a list of analysis events and the baseline event. Should correspond to the dict
analysis_evs = ['choice_ts']
baseline_ev = ['iti_ts']
evs = {'choice_ts': [-1.5, 1.5], # 1.5 seconds before choice, to 1.5 seconds after (full reward presentation)
       'iti_ts': [0.0, 0.5]}

In [ ]:
epochs_all_baseline = {}

In [ ]:
#baseline epoch - 
event = 'iti_ts'
epochs = epochs_all_evs[event]

good_chans = [x for x in epochs.ch_names if x not in epochs.info['bads']]
picks = [x for x in good_chans]

pow_struct = np.nan * np.ones([epochs._data.shape[0], 
                       epochs._data.shape[1], len(freqs), 
                       epochs._data.shape[-1]])

for ch_ix in np.arange(epochs._data.shape[1]): 
    ch_data = epochs._data[:, ch_ix:ch_ix+1, :]
    bad_epochs  = np.where(epochs.metadata[epochs.ch_names[ch_ix]].notnull())[0] # this is where detected IEDs are removed.
    good_epochs = np.delete(np.arange(ch_data.shape[0]), bad_epochs)
    ch_data = np.delete(ch_data, bad_epochs, axis=0)
    ch_pow = mne.time_frequency.tfr_array_morlet(ch_data, sfreq=epochs.info['sfreq'], 
                                        freqs=freqs, n_cycles=n_cycles, zero_mean=False, 
                                        use_fft=True, output='power', n_jobs=1)

    pow_struct[good_epochs, ch_ix, :, :] = ch_pow[:, 0, :, :]

temp_pow = mne.time_frequency.EpochsTFR(epochs.info, pow_struct, 
                                        epochs.times, freqs)
temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1]) # this is where you crop the buffer! 

epochs_all_baseline[event] = temp_pow

In [ ]:
power_epochs = {}

In [ ]:
event = 'choice_ts'

epochs = epochs_all_evs[event]

# Let's make sure we only do this for good channels
good_chans = [x for x in epochs.ch_names if x not in epochs.info['bads']]
picks = [x for x in good_chans]

pow_struct = np.nan * np.ones([epochs._data.shape[0], 
                       epochs._data.shape[1], len(freqs), 
                       epochs._data.shape[-1]])

for ch_ix in np.arange(epochs._data.shape[1]): 
    ch_data = epochs._data[:, ch_ix:ch_ix+1, :]
    bad_epochs  = np.where(epochs.metadata[epochs.ch_names[ch_ix]].notnull())[0] 
    good_epochs = np.delete(np.arange(ch_data.shape[0]), bad_epochs)
    ch_data = np.delete(ch_data, bad_epochs, axis=0) #this is where bad epochs for ch are deleted!!
    ch_pow = mne.time_frequency.tfr_array_morlet(ch_data, sfreq=epochs.info['sfreq'], 
                                        freqs=freqs, n_cycles=n_cycles, zero_mean=False, 
                                        use_fft=True, output='power', n_jobs=1)

    pow_struct[good_epochs, ch_ix, :, :] = ch_pow[:, 0, :, :]

temp_pow = mne.time_frequency.EpochsTFR(epochs.info, pow_struct, 
                                        epochs.times, freqs)

temp_pow.crop(tmin=evs[event][0], tmax=evs[event][1]) # this is where you crop the buffer! 


baseline_corrected_power = lfp_preprocess_utils.baseline_trialwise_TFR(data=temp_pow.data, 
                                                  baseline_mne=epochs_all_baseline['iti_ts'], 
                                                  mode='zscore', 
                                                  trialwise=False, 
                                                  baseline_only=True)


zpow = mne.time_frequency.EpochsTFR(epochs.info, baseline_corrected_power, 
                                temp_pow.times, freqs)

zpow.metadata = epochs_all_evs[event].metadata

power_epochs[event] = zpow


In [ ]:
power_epochs['choice_ts']

In [ ]:
power_epochs['choice_ts'].save(f'{neural_dir}/wm_pow_epochs_final-tfr.h5', overwrite=True)

Wavelet TFRs - all trials, every electrode

In [ ]:
event = 'choice_ts'
yticks = [4, 12, 30, 60, 90, 120, 150, 180, 200]
good_ch = [x for x in power_epochs[event].ch_names if '-' in x]


for ch in good_ch:
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    times = power_epochs[event].times
    plot_data = np.nanmean(np.nanmean(power_epochs[event].copy().pick_channels([ch]).data, axis=0), axis=0)

    im = ax.imshow(plot_data,
            extent=[times[0], times[-1], freqs[0], freqs[-1]], interpolation='Bicubic',
            aspect='auto', origin='lower', cmap='RdBu_r',vmin = -np.nanmax(np.abs(plot_data)), vmax = np.nanmax(np.abs(plot_data)))
    ax.set(yticks=yticks, xlabel='Time (s)', ylabel='Frequency',title=f'{ch} Encoding')
    ax.yaxis.set_tick_params(labelsize=8)
    fig.colorbar(im, ax=ax)
    plt.savefig(f'{save_dir}/{ch}.png', format='png', metadata=None,
    bbox_inches=None, pad_inches=0.1,
    facecolor='auto', edgecolor='auto',
    backend=None)
    plt.close()

Wavelet TFRs - REGION AVERAGED

In [ ]:
anat = pd.read_csv(f'{base_dir}/{subject}/Anat/{subject}_labels.csv')
anat['bin'].unique()

In [ ]:
anode_list = [x.split('-')[0] for x in epochs_all_evs['choice_ts'].ch_names]
elec_df = anat[anat.NMMlabel.str.lower().isin(anode_list)]

In [ ]:
region = 'ofc'
# band definitions for y-axis
yticks = [4, 8, 13, 30, 60, 120]

elecs_to_pick = elec_df.loc[elec_df['bin'] == region, 'NMMlabel'].str.lower() + '-'
elecs_to_pick = elecs_to_pick.tolist()

picks = []
for e in elecs_to_pick:
    picks_temp = list(filter(lambda s: e in s,  power_epochs['choice_ts'].info['ch_names']))[0]
    picks.append(picks_temp)

num_elecs = len(picks)
    
for event in analysis_evs:
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    times = power_epochs[event].times
    plot_data = np.nanmean(np.nanmean(power_epochs[event].copy().pick_channels(picks).data, axis=0), axis=0)

    im = ax.imshow(plot_data,extent=[times[0], times[-1], freqs[0], freqs[-1]], interpolation='Bicubic',aspect='auto', origin='lower', cmap='RdBu_r', vmin = -np.nanmax(np.abs(plot_data)), vmax = np.nanmax(np.abs(plot_data)))
    ax.set(yticks=yticks, xlabel='Time (s)', ylabel='Frequency', title = f'{region} - {num_elecs} electrodes')
    fig.colorbar(im, ax=ax)
    plt.savefig(f'{save_dir}/{region}.png', format='png', metadata=None,
    bbox_inches=None, pad_inches=0.1,
    facecolor='auto', edgecolor='auto',
    backend=None)
    plt.close()

With contrasts

In [ ]:
region = 'hippocampus'
# band definitions for y-axis
yticks = [4, 8, 13, 30, 60, 120]

conditions = ["(condition == 'hint')",
             "(condition == 'no_hint')"]

cond_name = 'Context'

elecs_to_pick = elec_df.loc[elec_df['bin'] == region, 'NMMlabel'].str.lower() + '-'
elecs_to_pick = elecs_to_pick.tolist()

picks = []
for e in elecs_to_pick:
    picks_temp = list(filter(lambda s: e in s,  power_epochs['choice_ts'].info['ch_names']))[0]
    picks.append(picks_temp)

    
for event in analysis_evs:
    fig, ax = plt.subplots(1, 2, figsize=(20, 6), dpi=300)
    for ix, cond in enumerate(conditions):
        times = power_epochs[event].times
        plot_data = np.nanmean(np.nanmean(power_epochs[event][cond].copy().pick_channels(picks).data, axis=0), axis=0)

        im = ax[ix].imshow(plot_data,
                    extent=[times[0], times[-1], freqs[0], freqs[-1]], interpolation='Bicubic',
                    aspect='auto', origin='lower', cmap='RdBu_r', vmin = -np.nanmax(np.abs(plot_data)), vmax = np.nanmax(np.abs(plot_data)))
        ax[ix].set(yticks=yticks, xlabel='Time (s)', ylabel='Frequency', title=f'{region}_{cond}')
        fig.colorbar(im, ax=ax[ix])

Save data for regressions and determine encoding elecs using permutation test

In [4]:
subject = "MS011"

In [5]:
# load power epochs and subset instructed trials
power_epochs = mne.time_frequency.read_tfrs(f'/Users/christinamaher/Desktop/fellowship_data/{subject}/Ephys/wm_pow_epochs_final-tfr.h5')
data = power_epochs[0].to_data_frame()
hfa_df = data.loc[data['freq'] >= 60]
hfa_df_melt = pd.melt(hfa_df, id_vars=["time", "freq","epoch","condition"],var_name="electrode", value_name="power")
average_by_epoch = hfa_df_melt.groupby(['time','epoch','electrode'])['power'].mean()
average_by_epoch.to_csv(f'/Users/christinamaher/Desktop/fellowship_data/{subject}/Ephys/average_by_epoch.csv')
regressors = power_epochs[0].metadata
regressors.to_csv(f'/Users/christinamaher/Desktop/fellowship_data/{subject}/Ephys/regressors.csv')

In [ ]:
permutation_df = pd.read_csv(f'/Users/christinamaher/Desktop/fellowship_data/{subject}/Ephys/average_by_epoch.csv')
electrode_list = permutation_df[permutation_df['electrode'].str.contains('-')]['electrode'].unique()

In [ ]:
def get_null_permutation(timeseries=None, win_len=200, slide_len=50):
    # Generate permuted timeseries
    surr = np.zeros((timeseries.shape))
    
    shuffles = np.random.randint(1, 300) # 3rd dimension of epoched data is timestamps
    surr_ts = np.roll(timeseries, shuffles, axis=1)
    np.random.shuffle(surr_ts)
    
    #surr = np.concatenate(surr_list)
    return surr_ts

In [ ]:
encoding_elec = []
electrode_name = []
for e in electrode_list:
    #electrode_data = power_epochs[0].copy().pick_channels([e]).data
    electrode_data = np.nanmean(np.nanmean(power_epochs[0].copy().pick_channels([e]).data, axis=0), axis=0)
    null_permutation = get_null_permutation(timeseries=electrode_data)
    F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test([electrode_data, null_permutation], out_type='mask', tail=0)
    
    # Create a boolean mask for values lower than 0.05
    mask = cluster_p_values < 0.001
    contains_value_lower_than_0_01 = np.any(mask)

    # Print the result
    if contains_value_lower_than_0_01:
        encoding = 1
    else:
        encoding = 0 
        
    encoding_elec.append(encoding) # save data so that encoding versus non-encoding electrodes can be subset
    electrode_name.append(e)

In [ ]:
# Zip the arrays together
zipped_data = list(zip(electrode_name, encoding_elec))

# Create a DataFrame
encoding_df = pd.DataFrame(zipped_data, columns=['name', 'encoding'])

encoding_df.to_csv(f'/Users/christinamaher/Desktop/fellowship_data/{subject}/Ephys/encoding_electrodes.csv')

In [4]:
power_epochs = mne.time_frequency.read_tfrs(fname='/Users/christinamaher/Desktop/fellowship_data/MS011/Ephys/wm_pow_epochs_final-tfr.h5')

In [6]:
region = 'ofc'

elec_df = pd.read_csv('/Users/christinamaher/Desktop/fellowship_data/MS011/Anat/MS011_labels.csv')

elecs_to_pick = elec_df.loc[elec_df['bin'] == region, 'label'].str.lower() + '-'
elecs_to_pick = elecs_to_pick.tolist()

picks = []
for e in elecs_to_pick:
    picks_temp = list(filter(lambda s: e in s,  power_epochs[0].info['ch_names']))
    picks.append(picks_temp)

picks = [item for sublist in picks for item in sublist]
picks = [picks]
picks

[['rmolf3-rmolf4', 'rmolf5-rmolf7']]

In [8]:
subj_id = 'MS011'
region = 'ofc'
analysis_evs = ['choice_ts']
freqs = np.logspace(*np.log10([2, 200]), num=30)
yticks = [4, 13, 30, 60, 120]



save_path = '/Users/christinamaher/Desktop/christinamaher/fellowship_data/MS011/Ephys'
# Get electrode df 

elec_df = pd.read_csv('/Users/christinamaher/Desktop/fellowship_data/MS011/Anat/MS011_labels.csv')

elecs_to_pick = elec_df.loc[elec_df['bin'] == region, 'label'].str.lower() + '-'
elecs_to_pick = elecs_to_pick.tolist()

picks = []
for e in elecs_to_pick:
    picks_temp = list(filter(lambda s: e in s,  power_epochs[0].info['ch_names']))
    picks.append(picks_temp)

picks = [item for sublist in picks for item in sublist][1]
picks = [picks]
    
conditions = ["(condition == 'no_hint')",
             "(condition == 'hint')"]

cond_name = 'Context'


for event in analysis_evs:
    
    # Average the data in each condition across channels 
    X = [np.nanmean(power_epochs[0][conditions[0]].copy().pick_channels(picks).data, axis=1), 
         np.nanmean(power_epochs[0][conditions[1]].copy().pick_channels(picks).data, axis=1)]
    
    F_obs, clusters, cluster_p_values, H0 = \
    mne.stats.permutation_cluster_test(X, n_permutations=500, out_type='mask', verbose=True)
    
    if any(cluster_p_values<=0.05):
        # Create new stats image with only significant clusters
        fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)

        times =  power_epochs[0][conditions[0]].times


        # Average the data in each condition across epochs for plotting
        evoked_power_1 = np.nanmean(X[0], axis=0)
        evoked_power_2 = np.nanmean(X[1], axis=0)
        evoked_power_contrast = evoked_power_1 - evoked_power_2
        signs = np.sign(evoked_power_contrast)

        F_obs_plot = np.nan * np.ones_like(F_obs)
        for c, p_val in zip(clusters, cluster_p_values):
            if p_val <= 0.05:
                F_obs_plot[c] = F_obs[c] * signs[c]

        ax.imshow(F_obs,
                  extent=[times[0], times[-1], freqs[0], freqs[-1]], interpolation = 'Bicubic',
                  aspect='auto', origin='lower', cmap='gray')
        max_F = np.nanmax(abs(F_obs_plot))
        ax.imshow(F_obs_plot,
                  extent=[times[0], times[-1], freqs[0], freqs[-1]],
                  aspect='auto', origin='lower', cmap='RdBu_r',
                  vmin=-max_F, vmax=max_F)

        ax.set(yticks=yticks, xlabel='Time (s)', ylabel='Frequency', title=f'{subj_id}_{region}_{cond_name}_{event}')
        
        # # ax.set_title(f'Induced power ({ch_name})')

  0%|          | Permuting : 0/499 [00:00<?,       ?it/s]